In [13]:
import os
from pathlib import Path
import numpy as np
import spectral
from skimage.filters import threshold_otsu
from skimage.morphology import remove_small_objects, remove_small_holes, binary_closing, disk
from skimage.measure import label, regionprops
from skimage.color import label2rgb
from scipy.ndimage import gaussian_filter
import matplotlib.pyplot as plt

def count_blueberries(bil_file, output_dir, min_size=500, max_size=10000, display=False):
    """
    Detects and counts blueberries in a hyperspectral image.

    Args:
    - bil_file (str): Path to the .bil file.
    - output_dir (str): Directory to save output files.
    - min_size (int): Minimum size of objects to count.
    - max_size (int): Maximum size of objects to count.
    - display (bool): Whether to display the processed images.

    Returns:
    - int: Number of detected blueberries.
    """
    # Load the hyperspectral image
    hdr_file = f"{bil_file}.hdr"
    hsi_image = spectral.open_image(hdr_file).load()

    # Convert to grayscale by summing along the spectral dimension
    grayscale_image = np.sum(hsi_image, axis=-1).astype(np.float32)

    # Normalize to 0-255 range for easier processing
    grayscale_image = (grayscale_image - np.min(grayscale_image)) / (np.max(grayscale_image) - np.min(grayscale_image))
    grayscale_image = (grayscale_image * 255).astype(np.uint8)

    # Apply Gaussian smoothing
    smoothed_image = gaussian_filter(grayscale_image, sigma=2)

    # Thresholding using Otsu's method
    threshold = threshold_otsu(smoothed_image)
    binary_mask = smoothed_image > threshold

    # Morphological cleaning
    binary_mask = remove_small_objects(binary_mask, min_size=min_size)
    binary_mask = remove_small_holes(binary_mask, area_threshold=min_size)
    binary_mask = binary_closing(binary_mask, disk(3))

    # Label connected components
    labeled_mask = label(binary_mask)
    regions = regionprops(labeled_mask)

    # Count valid blueberries based on size
    blueberry_count = 0
    for region in regions:
        if min_size <= region.area <= max_size:
            blueberry_count += 1

    # Save labeled mask
    output_path = os.path.join(output_dir, f"{Path(bil_file).stem}_labeled.png")
    Path(output_dir).mkdir(parents=True, exist_ok=True)
    label_overlay = label2rgb(labeled_mask, image=smoothed_image, bg_label=0)
    plt.imsave(output_path, label_overlay)

    # Optionally display results
    if display:
        plt.figure(figsize=(10, 5))
        plt.subplot(1, 3, 1)
        plt.title("Grayscale Image")
        plt.imshow(grayscale_image, cmap="gray")
        plt.axis("off")

        plt.subplot(1, 3, 2)
        plt.title("Binary Mask")
        plt.imshow(binary_mask, cmap="gray")
        plt.axis("off")

        plt.subplot(1, 3, 3)
        plt.title(f"Labeled Mask: {blueberry_count} Blueberries")
        plt.imshow(label_overlay)
        plt.axis("off")

        plt.tight_layout()
        plt.show()

    print(f"Processed: {bil_file} | Detected Blueberries: {blueberry_count}")
    return blueberry_count

def process_all_files(input_dir, output_dir, display=False):
    """
    Processes all .bil files in a directory and counts blueberries in each file.

    Args:
    - input_dir (str): Directory containing .bil files.
    - output_dir (str): Directory to save outputs.
    - display (bool): Whether to display the images during processing.
    """
    bil_files = [f for f in os.listdir(input_dir) if f.endswith(".bil")]
    total_blueberries = 0

    for bil_file in bil_files:
        bil_path = os.path.join(input_dir, bil_file)
        count = count_blueberries(bil_path, output_dir, display=display)
        total_blueberries += count

    print(f"Total Blueberries Detected Across All Files: {total_blueberries}")

# Example usage
input_directory = "/kaggle/input/hyperspectral-blueberries"
output_directory = "/kaggle/working/"

# Process all .bil files and count blueberries
process_all_files(input_directory, output_directory, display=False)

Processed: /kaggle/input/hyperspectral-blueberries/Bad Blueberries 1-42.bil | Detected Blueberries: 3
Processed: /kaggle/input/hyperspectral-blueberries/Bad Blueberries 127-168.bil | Detected Blueberries: 1
Processed: /kaggle/input/hyperspectral-blueberries/Good Blueberry 127-168.bil | Detected Blueberries: 0
Processed: /kaggle/input/hyperspectral-blueberries/WhiteReference.bil | Detected Blueberries: 0
Processed: /kaggle/input/hyperspectral-blueberries/Bad Blueberries 169-210.bil | Detected Blueberries: 0
Processed: /kaggle/input/hyperspectral-blueberries/Good Blueberry 1-42.bil | Detected Blueberries: 2
Processed: /kaggle/input/hyperspectral-blueberries/Good Blueberry 169-210.bil | Detected Blueberries: 0
Processed: /kaggle/input/hyperspectral-blueberries/Good Blueberry 85-126.bil | Detected Blueberries: 0
Processed: /kaggle/input/hyperspectral-blueberries/Bad Blueberry 85-126.bil | Detected Blueberries: 2
Processed: /kaggle/input/hyperspectral-blueberries/Bad Blueberry 43-84.bil | D